## One file, id, lang, text, level, questionid?, len (fixed), tokens

In [1]:
import os
import nltk
import re
import numpy as np
import regex
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
import pandas as pd
import string
import seaborn as sns

In [2]:
loose=pd.read_csv('original-sheets\Responses-Loose.csv')
student_info=pd.read_csv('corpus-files\student_information.csv')
course=pd.read_csv('corpus-files\course.csv')
answer_df=pd.read_csv('answer.csv',index_col = 'answer_id', low_memory=False)


In [3]:
#dropping to only relevant columns
loose=loose.drop(columns=['3Langs','Item','gender','course_id','class_id','version','text1_len','text2_len','text1',
                         'text2 (line breaks/extra spaces removed, spaces added to reach 60)','Judgement','Notes'])

In [4]:
course_=course.drop(columns=['class_id','semester','section'])

student=student_info.drop(columns=['birth_year','language_used_at_home','non_native_language_1','yrs_of_study_lang1',
                                  'study_in_classroom_lang1','ways_of_study_lang1','non_native_language_2','study_in_classroom_lang2',
                                  'ways_of_study_lang2','non_native_language_3','yrs_of_study_lang3','study_in_classroom_lang3',
                                  'ways_of_study_lang3','yrs_of_english_learning','yrs_in_english_environment','yrs_of_study_lang2'])
answer_=answer_df[['question_id','anon_id','course_id','version','text_len','text','tokens']].reset_index()

In [5]:
answer_

,answer_id,question_id,anon_id,course_id,version,text_len,text,tokens
0,1,5,eq0,149,1,177,I met my friend Nife while I was studying in a...,"['I', 'met', 'my', 'friend', 'Nife', 'while', ..."
1,2,5,am8,149,1,137,"Ten years ago, I met a women on the train betw...","['Ten', 'years', 'ago', ',', 'I', 'met', 'a', ..."
2,3,12,dk5,115,1,63,In my country we usually don't use tea bags. F...,"['In', 'my', 'country', 'we', 'usually', 'do',..."
3,4,13,dk5,115,1,6,I organized the instructions by time.,"['I', 'organized', 'the', 'instructions', 'by'..."
4,5,12,ad1,115,1,59,"First, prepare a port, loose tea, and cup.\nSe...","['First', ',', 'prepare', 'a', 'port', ',', 'l..."
...,...,...,...,...,...,...,...,...
46312,48416,6139,dr8,1050,1,439,Many people enjoy a game in their free time. B...,"['Many', 'people', 'enjoy', 'a', 'game', 'in',..."
46313,48417,6137,fv1,1045,1,478,\n Marijuana Legalization 7/22/12\n Writing 4...,"['Marijuana', 'Legalization', '7', '/', '22', ..."
46314,48418,6137,fo1,1045,1,408,Some patients are suffering from their illnes...,"['Some', 'patients', 'are', 'suffering', 'from..."
46315,48419,6119,ge8,1033,1,110,My house looks amazing and modern. I decorated...,"['My', 'house', 'looks', 'amazing', 'and', 'mo..."


In [6]:
answer_.dtypes

answer_id      object
question_id    object
anon_id        object
course_id      object
version        object
text_len       object
text           object
tokens         object
dtype: object

In [7]:
#This function removes all of the rows that contain a string in the passed column
def rmStr (df, col):
    row_index=0
    row_ind=[]
    for i in df[col]:
        try:
            int(i)
        except:
            row_ind.append(row_index)
        row_index+=1
    return df.drop(labels=row_ind,axis=0)

In [8]:
col='course_id'
answer=rmStr(answer_,col)

In [9]:
merge_ans=answer[['anon_id','course_id']] #sets answer df up for merging 
student_merge=student.drop(columns=['course_history']) #set student df up for merging

student_ans=student_merge.merge(answer,on='anon_id').astype({'course_id':'int64'}) #merges student and answers

stu_ans_crs_=student_ans.merge(course_, on='course_id').drop(columns=['age','course_id','version','tokens']) #merges the student-answers df with course df

In [10]:
# function that finds the unique values in a list
def get_uniques(x):
    return list(x.unique())

# whole dataset

In [11]:
stu_ans_crs_.dtypes

anon_id            object
gender             object
native_language    object
answer_id          object
question_id        object
text_len           object
text               object
level_id            int64
dtype: object

In [12]:
stu_ans_crs_.isnull().sum()

anon_id            0
gender             0
native_language    0
answer_id          0
question_id        0
text_len           0
text               0
level_id           0
dtype: int64

In [13]:
# adding spaces after punctuation if needed

index=0
stu_ans_crs=stu_ans_crs_.copy()
stu_ans_crs['processed_text']=pd.NaT
stu_ans_crs['wordtype_len']=pd.NaT
words=[]
for text in stu_ans_crs.text:
    uwords=[]
    text=text.replace('\n',' ')
    nopunc=text.translate(str.maketrans('','',string.punctuation))
    nopunc=nopunc.lower()
    stu_ans_crs.at[index,'processed_text']=nopunc
    words=nopunc.strip().split(" ")
    for word in words:
        if word=='':
            words.remove(word)
        elif word not in uwords:
            uwords.append(word)
    stu_ans_crs.at[index,'text_len']=len(words)
    stu_ans_crs.at[index,'wordtype_len']=len(uwords)
    index+=1

In [14]:
#dropping all rows where the answer is less than 60 words
index=0
ind_list=[]
for i in stu_ans_crs.text_len:
    txtlen=int(i)
    if txtlen < 60:
        ind_list.append(index)
    index+=1

stu_ans_crs=stu_ans_crs.drop(labels=ind_list,axis=0).reset_index()

In [15]:
stu_ans_crs

,index,anon_id,gender,native_language,answer_id,question_id,text_len,text,level_id,processed_text,wordtype_len
0,0,do6,Female,Russian,150,4,299,Some people prefer eat out and some like doing...,5,some people prefer eat out and some like doing...,125
1,1,do6,Female,Russian,1221,97,288,My opinion is that a person does need educatio...,5,my opinion is that a person does need educatio...,112
2,2,do6,Female,Russian,1957,189,317,There are two national rooms in the Cathedral ...,5,there are two national rooms in the cathedral ...,149
3,3,do6,Female,Russian,2164,190,459,There are two nation rooms in the Cathedral of...,5,there are two nation rooms in the cathedral of...,187
4,4,bv5,Male,Arabic,151,4,311,"""Not all learning takes place in the classroom...",5,not all learning takes place in the classroom ...,136
...,...,...,...,...,...,...,...,...,...,...,...
17144,46214,bh8,Male,Japanese,48034,6087,107,I received my Medical Doctor license in Japan ...,3,i received my medical doctor license in japan ...,54
17145,46215,bh8,Male,Japanese,48293,6119,138,I introduce my ideal home to you quickly. This...,3,i introduce my ideal home to you quickly this ...,88
17146,46217,cz3,Female,Korean,47451,6027,102,I will put 5 items in time capsule such as som...,3,i will put 5 items in time capsule such as som...,67
17147,46218,cz3,Female,Korean,48001,6087,104,I have studied Visual Design since 2002. I gra...,3,i have studied visual design since 2002 i grad...,57


In [16]:
sac_lvls=stu_ans_crs.drop(axis=1,columns=['index','gender'])
slr=sac_lvls

sac_lvls_rkc=slr.query('native_language=="Korean" or native_language=="Spanish" or native_language=="Chinese"').reset_index().drop(columns=['index'])

sac_lvls_rkc

,anon_id,native_language,answer_id,question_id,text_len,text,level_id,processed_text,wordtype_len
0,ax4,Chinese,954,48,479,The Effects on Students Who Have a Part-time J...,5,the effects on students who have a parttime jo...,166
1,ax4,Chinese,1219,97,259,There are many different kinds of people in th...,5,there are many different kinds of people in th...,105
2,ax4,Chinese,1257,74,548,Should the Death Penalty Exist\nThe death pena...,5,should the death penalty exist the death penal...,191
3,ax4,Chinese,1260,119,576,Should the Death Penalty Exist?\nThe death pen...,5,should the death penalty exist the death penal...,213
4,ax4,Chinese,2162,190,180,The nationality rooms built in the Cathedral o...,5,the nationality rooms built in the cathedral o...,90
...,...,...,...,...,...,...,...,...,...
7575,ew6,Chinese,48035,6087,111,I participated student organization in univers...,3,i participated student organization in univers...,66
7576,ew6,Chinese,48338,6119,140,My ideal home is big and modern. It is located...,3,my ideal home is big and modern it is located ...,84
7577,cz3,Korean,47451,6027,102,I will put 5 items in time capsule such as som...,3,i will put 5 items in time capsule such as som...,67
7578,cz3,Korean,48001,6087,104,I have studied Visual Design since 2002. I gra...,3,i have studied visual design since 2002 i grad...,57


In [17]:
filtered=sac_lvls_rkc.query('level_id==5 or level_id==4 or level_id==3').reset_index()

In [18]:
filtered

,index,anon_id,native_language,answer_id,question_id,text_len,text,level_id,processed_text,wordtype_len
0,0,ax4,Chinese,954,48,479,The Effects on Students Who Have a Part-time J...,5,the effects on students who have a parttime jo...,166
1,1,ax4,Chinese,1219,97,259,There are many different kinds of people in th...,5,there are many different kinds of people in th...,105
2,2,ax4,Chinese,1257,74,548,Should the Death Penalty Exist\nThe death pena...,5,should the death penalty exist the death penal...,191
3,3,ax4,Chinese,1260,119,576,Should the Death Penalty Exist?\nThe death pen...,5,should the death penalty exist the death penal...,213
4,4,ax4,Chinese,2162,190,180,The nationality rooms built in the Cathedral o...,5,the nationality rooms built in the cathedral o...,90
...,...,...,...,...,...,...,...,...,...,...
7534,7575,ew6,Chinese,48035,6087,111,I participated student organization in univers...,3,i participated student organization in univers...,66
7535,7576,ew6,Chinese,48338,6119,140,My ideal home is big and modern. It is located...,3,my ideal home is big and modern it is located ...,84
7536,7577,cz3,Korean,47451,6027,102,I will put 5 items in time capsule such as som...,3,i will put 5 items in time capsule such as som...,67
7537,7578,cz3,Korean,48001,6087,104,I have studied Visual Design since 2002. I gra...,3,i have studied visual design since 2002 i grad...,57


In [19]:
whole_dataset=filtered.drop(axis=1, columns=['index'])
whole_dataset

,anon_id,native_language,answer_id,question_id,text_len,text,level_id,processed_text,wordtype_len
0,ax4,Chinese,954,48,479,The Effects on Students Who Have a Part-time J...,5,the effects on students who have a parttime jo...,166
1,ax4,Chinese,1219,97,259,There are many different kinds of people in th...,5,there are many different kinds of people in th...,105
2,ax4,Chinese,1257,74,548,Should the Death Penalty Exist\nThe death pena...,5,should the death penalty exist the death penal...,191
3,ax4,Chinese,1260,119,576,Should the Death Penalty Exist?\nThe death pen...,5,should the death penalty exist the death penal...,213
4,ax4,Chinese,2162,190,180,The nationality rooms built in the Cathedral o...,5,the nationality rooms built in the cathedral o...,90
...,...,...,...,...,...,...,...,...,...
7534,ew6,Chinese,48035,6087,111,I participated student organization in univers...,3,i participated student organization in univers...,66
7535,ew6,Chinese,48338,6119,140,My ideal home is big and modern. It is located...,3,my ideal home is big and modern it is located ...,84
7536,cz3,Korean,47451,6027,102,I will put 5 items in time capsule such as som...,3,i will put 5 items in time capsule such as som...,67
7537,cz3,Korean,48001,6087,104,I have studied Visual Design since 2002. I gra...,3,i have studied visual design since 2002 i grad...,57


# Loose

In [20]:
drop_loose=loose.dropna()

sorted_loose=drop_loose.sort_values(by=['anon_id']).reset_index(drop=True)

In [21]:
sorted_loose

,answer_id,anon_id,L1,level_id,question_id,text3_len,text3 (edits made to fix word counts)
0,34686.0,aa0,Spanish,5.0,4497.0,109.0,"Barber, chef by profession, but an expert on a..."
1,32129.0,aa0,Spanish,5.0,4199.0,190.0,"The article ""English as Co star"" support the f..."
2,33302.0,aa0,Spanish,5.0,4346.0,193.0,"In this article the authors Goleman, Kaufman a..."
3,33308.0,aa0,Spanish,5.0,4347.0,170.0,"Flow in the sense expressed in the text ""The C..."
4,32572.0,aa0,Spanish,5.0,4260.0,85.0,"Bill Gates, in this conference, explained two ..."
...,...,...,...,...,...,...,...
1805,32406.0,ha2,Korean,5.0,4273.0,217.0,Pittsburgh had a big snowstorm recently. A lot...
1806,32326.0,ha2,Korean,5.0,4256.0,170.0,"Korea, which is connected to China, belongs to..."
1807,32077.0,ha2,Korean,5.0,4197.0,195.0,Koko is a big female gorilla who was born in S...
1808,29631.0,ha2,Korean,4.0,3943.0,283.0,A natural disaster is the effect of the natura...


In [22]:
index=0
textcount_loose=sorted_loose.copy()

textcount_loose['processed_text']=pd.NaT
textcount_loose['wordtype_len']=pd.NaT
words=[]
for text in textcount_loose['text3 (edits made to fix word counts)']:
    uwords=[]
    text=str(text).replace('\n',' ')
    nopunc=text.translate(str.maketrans('','',string.punctuation))
    nopunc=nopunc.lower()
    textcount_loose.at[index,'processed_text']=nopunc
    words=nopunc.strip().split(" ")
    for word in words:
        if word=='':
            words.remove(word)
        elif word not in uwords:
            uwords.append(word)
    textcount_loose.at[index,'text3_len']=len(words)
    textcount_loose.at[index,'wordtype_len']=len(uwords)
    index+=1

In [23]:
textcount_loose

,answer_id,anon_id,L1,level_id,question_id,text3_len,text3 (edits made to fix word counts),processed_text,wordtype_len
0,34686.0,aa0,Spanish,5.0,4497.0,109.0,"Barber, chef by profession, but an expert on a...",barber chef by profession but an expert on agr...,79
1,32129.0,aa0,Spanish,5.0,4199.0,190.0,"The article ""English as Co star"" support the f...",the article english as co star support the fac...,117
2,33302.0,aa0,Spanish,5.0,4346.0,193.0,"In this article the authors Goleman, Kaufman a...",in this article the authors goleman kaufman an...,122
3,33308.0,aa0,Spanish,5.0,4347.0,170.0,"Flow in the sense expressed in the text ""The C...",flow in the sense expressed in the text the cr...,103
4,32572.0,aa0,Spanish,5.0,4260.0,85.0,"Bill Gates, in this conference, explained two ...",bill gates in this conference explained two bi...,66
...,...,...,...,...,...,...,...,...,...
1805,32406.0,ha2,Korean,5.0,4273.0,217.0,Pittsburgh had a big snowstorm recently. A lot...,pittsburgh had a big snowstorm recently a lot ...,129
1806,32326.0,ha2,Korean,5.0,4256.0,170.0,"Korea, which is connected to China, belongs to...",korea which is connected to china belongs to t...,101
1807,32077.0,ha2,Korean,5.0,4197.0,195.0,Koko is a big female gorilla who was born in S...,koko is a big female gorilla who was born in s...,115
1808,29631.0,ha2,Korean,4.0,3943.0,283.0,A natural disaster is the effect of the natura...,a natural disaster is the effect of the natura...,135


# Keeping only revised entry

In [24]:
test=whole_dataset.copy().reset_index()
test

,index,anon_id,native_language,answer_id,question_id,text_len,text,level_id,processed_text,wordtype_len
0,0,ax4,Chinese,954,48,479,The Effects on Students Who Have a Part-time J...,5,the effects on students who have a parttime jo...,166
1,1,ax4,Chinese,1219,97,259,There are many different kinds of people in th...,5,there are many different kinds of people in th...,105
2,2,ax4,Chinese,1257,74,548,Should the Death Penalty Exist\nThe death pena...,5,should the death penalty exist the death penal...,191
3,3,ax4,Chinese,1260,119,576,Should the Death Penalty Exist?\nThe death pen...,5,should the death penalty exist the death penal...,213
4,4,ax4,Chinese,2162,190,180,The nationality rooms built in the Cathedral o...,5,the nationality rooms built in the cathedral o...,90
...,...,...,...,...,...,...,...,...,...,...
7534,7534,ew6,Chinese,48035,6087,111,I participated student organization in univers...,3,i participated student organization in univers...,66
7535,7535,ew6,Chinese,48338,6119,140,My ideal home is big and modern. It is located...,3,my ideal home is big and modern it is located ...,84
7536,7536,cz3,Korean,47451,6027,102,I will put 5 items in time capsule such as som...,3,i will put 5 items in time capsule such as som...,67
7537,7537,cz3,Korean,48001,6087,104,I have studied Visual Design since 2002. I gra...,3,i have studied visual design since 2002 i grad...,57


In [25]:
i=0
drop=[]
while i<len(test)-1:
    anon=test.at[i,'anon_id']
    n_anon=test.at[i+1,'anon_id']
    q_id=test.at[i,'question_id']
    nq_id=test.at[i+1,'question_id']
    if (anon==n_anon and q_id==nq_id):
        drop.append(i)
    i+=1
        
len(drop)
final=test.drop(drop)
final

,index,anon_id,native_language,answer_id,question_id,text_len,text,level_id,processed_text,wordtype_len
0,0,ax4,Chinese,954,48,479,The Effects on Students Who Have a Part-time J...,5,the effects on students who have a parttime jo...,166
1,1,ax4,Chinese,1219,97,259,There are many different kinds of people in th...,5,there are many different kinds of people in th...,105
2,2,ax4,Chinese,1257,74,548,Should the Death Penalty Exist\nThe death pena...,5,should the death penalty exist the death penal...,191
3,3,ax4,Chinese,1260,119,576,Should the Death Penalty Exist?\nThe death pen...,5,should the death penalty exist the death penal...,213
4,4,ax4,Chinese,2162,190,180,The nationality rooms built in the Cathedral o...,5,the nationality rooms built in the cathedral o...,90
...,...,...,...,...,...,...,...,...,...,...
7534,7534,ew6,Chinese,48035,6087,111,I participated student organization in univers...,3,i participated student organization in univers...,66
7535,7535,ew6,Chinese,48338,6119,140,My ideal home is big and modern. It is located...,3,my ideal home is big and modern it is located ...,84
7536,7536,cz3,Korean,47451,6027,102,I will put 5 items in time capsule such as som...,3,i will put 5 items in time capsule such as som...,67
7537,7537,cz3,Korean,48001,6087,104,I have studied Visual Design since 2002. I gra...,3,i have studied visual design since 2002 i grad...,57


In [26]:
no_dup_loose=textcount_loose.copy().reset_index()
i=0
drop=[]
while i<len(no_dup_loose)-1:
    anon=no_dup_loose.at[i,'anon_id']
    n_anon=no_dup_loose.at[i+1,'anon_id']
    q_id=no_dup_loose.at[i,'question_id']
    nq_id=no_dup_loose.at[i+1,'question_id']
    if (anon==n_anon and q_id==nq_id):
        drop.append(i)
    i+=1
        
len(drop)
final_loose=no_dup_loose.drop(drop)
final_loose

,index,answer_id,anon_id,L1,level_id,question_id,text3_len,text3 (edits made to fix word counts),processed_text,wordtype_len
0,0,34686.0,aa0,Spanish,5.0,4497.0,109.0,"Barber, chef by profession, but an expert on a...",barber chef by profession but an expert on agr...,79
1,1,32129.0,aa0,Spanish,5.0,4199.0,190.0,"The article ""English as Co star"" support the f...",the article english as co star support the fac...,117
2,2,33302.0,aa0,Spanish,5.0,4346.0,193.0,"In this article the authors Goleman, Kaufman a...",in this article the authors goleman kaufman an...,122
3,3,33308.0,aa0,Spanish,5.0,4347.0,170.0,"Flow in the sense expressed in the text ""The C...",flow in the sense expressed in the text the cr...,103
4,4,32572.0,aa0,Spanish,5.0,4260.0,85.0,"Bill Gates, in this conference, explained two ...",bill gates in this conference explained two bi...,66
...,...,...,...,...,...,...,...,...,...,...
1805,1805,32406.0,ha2,Korean,5.0,4273.0,217.0,Pittsburgh had a big snowstorm recently. A lot...,pittsburgh had a big snowstorm recently a lot ...,129
1806,1806,32326.0,ha2,Korean,5.0,4256.0,170.0,"Korea, which is connected to China, belongs to...",korea which is connected to china belongs to t...,101
1807,1807,32077.0,ha2,Korean,5.0,4197.0,195.0,Koko is a big female gorilla who was born in S...,koko is a big female gorilla who was born in s...,115
1808,1808,29631.0,ha2,Korean,4.0,3943.0,283.0,A natural disaster is the effect of the natura...,a natural disaster is the effect of the natura...,135


In [27]:
drop

[230, 936]

# Averages per Participant

In [28]:
# only uses whole dataset
final

,index,anon_id,native_language,answer_id,question_id,text_len,text,level_id,processed_text,wordtype_len
0,0,ax4,Chinese,954,48,479,The Effects on Students Who Have a Part-time J...,5,the effects on students who have a parttime jo...,166
1,1,ax4,Chinese,1219,97,259,There are many different kinds of people in th...,5,there are many different kinds of people in th...,105
2,2,ax4,Chinese,1257,74,548,Should the Death Penalty Exist\nThe death pena...,5,should the death penalty exist the death penal...,191
3,3,ax4,Chinese,1260,119,576,Should the Death Penalty Exist?\nThe death pen...,5,should the death penalty exist the death penal...,213
4,4,ax4,Chinese,2162,190,180,The nationality rooms built in the Cathedral o...,5,the nationality rooms built in the cathedral o...,90
...,...,...,...,...,...,...,...,...,...,...
7534,7534,ew6,Chinese,48035,6087,111,I participated student organization in univers...,3,i participated student organization in univers...,66
7535,7535,ew6,Chinese,48338,6119,140,My ideal home is big and modern. It is located...,3,my ideal home is big and modern it is located ...,84
7536,7536,cz3,Korean,47451,6027,102,I will put 5 items in time capsule such as som...,3,i will put 5 items in time capsule such as som...,67
7537,7537,cz3,Korean,48001,6087,104,I have studied Visual Design since 2002. I gra...,3,i have studied visual design since 2002 i grad...,57


In [29]:
final.dtypes

index               int64
anon_id            object
native_language    object
answer_id          object
question_id        object
text_len           object
text               object
level_id            int64
processed_text     object
wordtype_len       object
dtype: object

In [30]:
test=final.groupby(['anon_id']).mean()
test

,index,level_id
anon_id,,
aa0,5728.764706,5.000000
aa1,4745.444444,4.000000
aa3,7464.500000,4.000000
aa8,2108.250000,4.535714
aa9,2034.500000,3.450000
...,...,...
ha0,3122.333333,5.000000
ha2,5833.190476,4.690476
ha6,2962.192308,3.307692


In [31]:
sort=final.sort_values(by=['anon_id']).reset_index(drop=True)
sort['Len Avg L3']=0.0
sort['Len Avg L4']=0.0
sort['Len Avg L5']=0.0
sort['Type Avg L3']=0.0
sort['Type Avg L4']=0.0
sort['Type Avg L5']=0.0

In [32]:
sort.dtypes

index                int64
anon_id             object
native_language     object
answer_id           object
question_id         object
text_len            object
text                object
level_id             int64
processed_text      object
wordtype_len        object
Len Avg L3         float64
Len Avg L4         float64
Len Avg L5         float64
Type Avg L3        float64
Type Avg L4        float64
Type Avg L5        float64
dtype: object

In [33]:
sort

,index,anon_id,native_language,answer_id,question_id,text_len,text,level_id,processed_text,wordtype_len,Len Avg L3,Len Avg L4,Len Avg L5,Type Avg L3,Type Avg L4,Type Avg L5
0,5859,aa0,Spanish,35603,4607,157,Of course sport star earn much money. Once a s...,5,of course sport star earn much money once a sp...,99,0.0,0.0,0.0,0.0,0.0,0.0
1,6245,aa0,Spanish,33345,4339,163,Summary \nHans Rosling on HIV: New Facts and S...,5,summary hans rosling on hiv new facts and stu...,103,0.0,0.0,0.0,0.0,0.0,0.0
2,6246,aa0,Spanish,34686,4497,109,"Barber, chef by profession, but an expert on a...",5,barber chef by profession but an expert on agr...,79,0.0,0.0,0.0,0.0,0.0,0.0
3,5860,aa0,Spanish,35619,4608,208,The last time I saw a compassionate act was in...,5,the last time i saw a compassionate act was in...,119,0.0,0.0,0.0,0.0,0.0,0.0
4,6247,aa0,Spanish,35369,4602,139,The Skywalker story in Star War episode IV fix...,5,the skywalker story in star war episode iv fix...,95,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6672,6259,hc1,Korean,39612,5097,126,\n Alcohol consumption in the U.S has been on ...,5,alcohol consumption in the us has been on th...,85,0.0,0.0,0.0,0.0,0.0,0.0
6673,6257,hc1,Korean,38045,4910,550,\n\n\nName: Jinny \n\nDate: 09/25/2010 \n\nWri...,5,name jinny date 09252010 writing class ...,254,0.0,0.0,0.0,0.0,0.0,0.0
6674,6261,hc1,Korean,41023,5238,282,"Since I was young, I have had inherently a lot...",5,since i was young i have had inherently a lot ...,138,0.0,0.0,0.0,0.0,0.0,0.0
6675,6258,hc1,Korean,38433,4984,306,How many people use English as a mother langua...,5,how many people use english as a mother langua...,158,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
i=0
while i <len(sort):
    level=sort.at[i,'level_id']
    length=sort.at[i,'text_len']
    wordtype=sort.at[i,'wordtype_len']
    if level==5:
        sort.at[i,'Len Avg L5']=length
        sort.at[i,'Type Avg L5']=wordtype
    elif level==4:
        sort.at[i,'Len Avg L4']=length
        sort.at[i,'Type Avg L4']=wordtype
    elif level==3:
        sort.at[i,'Len Avg L3']=length
        sort.at[i,'Type Avg L3']=wordtype
    i+=1
sort

,index,anon_id,native_language,answer_id,question_id,text_len,text,level_id,processed_text,wordtype_len,Len Avg L3,Len Avg L4,Len Avg L5,Type Avg L3,Type Avg L4,Type Avg L5
0,5859,aa0,Spanish,35603,4607,157,Of course sport star earn much money. Once a s...,5,of course sport star earn much money once a sp...,99,0.0,0.0,157.0,0.0,0.0,99.0
1,6245,aa0,Spanish,33345,4339,163,Summary \nHans Rosling on HIV: New Facts and S...,5,summary hans rosling on hiv new facts and stu...,103,0.0,0.0,163.0,0.0,0.0,103.0
2,6246,aa0,Spanish,34686,4497,109,"Barber, chef by profession, but an expert on a...",5,barber chef by profession but an expert on agr...,79,0.0,0.0,109.0,0.0,0.0,79.0
3,5860,aa0,Spanish,35619,4608,208,The last time I saw a compassionate act was in...,5,the last time i saw a compassionate act was in...,119,0.0,0.0,208.0,0.0,0.0,119.0
4,6247,aa0,Spanish,35369,4602,139,The Skywalker story in Star War episode IV fix...,5,the skywalker story in star war episode iv fix...,95,0.0,0.0,139.0,0.0,0.0,95.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6672,6259,hc1,Korean,39612,5097,126,\n Alcohol consumption in the U.S has been on ...,5,alcohol consumption in the us has been on th...,85,0.0,0.0,126.0,0.0,0.0,85.0
6673,6257,hc1,Korean,38045,4910,550,\n\n\nName: Jinny \n\nDate: 09/25/2010 \n\nWri...,5,name jinny date 09252010 writing class ...,254,0.0,0.0,550.0,0.0,0.0,254.0
6674,6261,hc1,Korean,41023,5238,282,"Since I was young, I have had inherently a lot...",5,since i was young i have had inherently a lot ...,138,0.0,0.0,282.0,0.0,0.0,138.0
6675,6258,hc1,Korean,38433,4984,306,How many people use English as a mother langua...,5,how many people use english as a mother langua...,158,0.0,0.0,306.0,0.0,0.0,158.0


In [35]:
avg=sort.groupby(['anon_id']).mean().reset_index()
avg.drop(columns=['index','level_id'],inplace=True)

In [36]:
avg

,anon_id,Len Avg L3,Len Avg L4,Len Avg L5,Type Avg L3,Type Avg L4,Type Avg L5
0,aa0,0.000000,0.000000,208.235294,0.000000,0.000000,120.588235
1,aa1,0.000000,159.944444,0.000000,0.000000,85.222222,0.000000
2,aa3,0.000000,144.500000,0.000000,0.000000,89.375000,0.000000
3,aa8,0.000000,62.857143,250.142857,0.000000,34.642857,92.142857
4,aa9,54.550000,58.900000,0.000000,32.700000,31.950000,0.000000
...,...,...,...,...,...,...,...
468,ha0,0.000000,0.000000,289.666667,0.000000,0.000000,140.000000
469,ha2,0.000000,39.452381,271.500000,0.000000,21.309524,118.714286
470,ha6,94.769231,45.923077,0.000000,55.038462,26.538462,0.000000
471,hb4,56.277778,69.222222,88.500000,26.444444,35.944444,42.805556


In [37]:
lang=sort[['anon_id','native_language']]
lang.head()

,anon_id,native_language
0,aa0,Spanish
1,aa0,Spanish
2,aa0,Spanish
3,aa0,Spanish
4,aa0,Spanish


In [38]:
i=0
index=0
avg['lang']=""
while index<len(avg):
    anon=lang.at[i,'anon_id']
    avg_anon=avg.at[index,'anon_id']
    if anon==avg_anon:
        avg.at[index,'lang']=lang.at[i,'native_language']
        index+=1
    i+=1

In [39]:
avg

,anon_id,Len Avg L3,Len Avg L4,Len Avg L5,Type Avg L3,Type Avg L4,Type Avg L5,lang
0,aa0,0.000000,0.000000,208.235294,0.000000,0.000000,120.588235,Spanish
1,aa1,0.000000,159.944444,0.000000,0.000000,85.222222,0.000000,Chinese
2,aa3,0.000000,144.500000,0.000000,0.000000,89.375000,0.000000,Chinese
3,aa8,0.000000,62.857143,250.142857,0.000000,34.642857,92.142857,Korean
4,aa9,54.550000,58.900000,0.000000,32.700000,31.950000,0.000000,Korean
...,...,...,...,...,...,...,...,...
468,ha0,0.000000,0.000000,289.666667,0.000000,0.000000,140.000000,Korean
469,ha2,0.000000,39.452381,271.500000,0.000000,21.309524,118.714286,Korean
470,ha6,94.769231,45.923077,0.000000,55.038462,26.538462,0.000000,Korean
471,hb4,56.277778,69.222222,88.500000,26.444444,35.944444,42.805556,Korean


# Counts

In [40]:
final.groupby(['native_language']).agg({'native_language':'count'})

,native_language
native_language,
Chinese,2852
Korean,3114
Spanish,711


In [41]:
sp=pd.read_csv('C:/Users/Hanna/Documents/CSV/same_probe.csv')
sp.groupby(['L1']).agg({'L1':'count'})

,L1
L1,
Chinese,647
Korean,770
Spanish,391


In [42]:
sp

,L1,question_id,anon_id,text3_len,wordtype_len,level_id,answer_id,text3 (edits made to fix word counts),processed_text
0,Chinese,17,aj8,114,74,5,141,"Today, I am going to describe one of my classm...",today i am going to describe one of my classma...
1,Korean,17,at8,105,60,5,135,my friend is ANON_NAME_0. she is a my ELI frie...,my friend is anonname0 she is a my eli friend ...
2,Korean,17,az2,128,72,5,133,"When I was in Germany, I met a friend who was ...",when i was in germany i met a friend who was f...
3,Chinese,17,az8,95,67,5,143,My niece is 3 years old who is my younger brot...,my niece is 3 years old who is my younger brot...
4,Spanish,17,ea7,99,61,5,136,My mother is the most beautiful person that I ...,my mother is the most beautiful person that i ...
...,...,...,...,...,...,...,...,...,...
1803,Korean,6070,ec1,138,80,4,47845,My best friend is Sohyun who graduated from th...,my best friend is sohyun who graduated from th...
1804,Chinese,6070,fk2,84,57,4,47868,Annie is my best friend who is also my schoolm...,annie is my best friend who is also my schoolm...
1805,Spanish,6105,bl3,146,93,4,48252,This quotation means that the majority of peop...,this quotation means that the majority of peop...
1806,Chinese,6105,bv8,131,73,4,48246,The quote that talks about a true friend is so...,the quote that talks about a true friend is so...


In [43]:
sp=pd.read_csv('C:/Users/Hanna/Documents/CSV/same_probe.csv')
print('Loose Dataset: ')
sp.groupby(['L1','level_id']).agg({'L1':'count'})

Loose Dataset: 


L1
L1      level_id     
Chinese 3          52
        4         357
        5         238
Korean  3          98
        4         444
        5         228
Spanish 3          60
        4         207
        5         124

In [44]:
print("Whole Dataset: ")
final.groupby(['native_language','level_id']).agg({'native_language':'count'})

Whole Dataset: 


native_language
native_language level_id                 
Chinese         3                     461
                4                    1359
                5                    1032
Korean          3                     552
                4                    1494
                5                    1068
Spanish         3                     149
                4                     332
                5                     230

In [45]:
print("Whole Dataset Level counts:")
final.groupby(['level_id']).agg({'level_id':'count'})

Whole Dataset Level counts:


,level_id
level_id,
3,1162
4,3185
5,2330


In [46]:
print("Loose Dataset Level counts:")
sp.groupby(['level_id']).agg({'level_id':'count'})

Loose Dataset Level counts:


,level_id
level_id,
3,210
4,1008
5,590


# Exporting CSV

In [47]:
avg.to_csv('C:/Users/Hanna/Documents/CSV/participant_averages.csv')

In [48]:
final.to_csv('C:/Users/Hanna/Documents/CSV/whole_no_duplicates.csv')

In [49]:
textcount_loose.to_csv('C:/Users/Hanna/Documents/CSV/loose.csv')

In [58]:
final_loose.to_csv('C:/Users\Hanna\Documents\GitHub\LRDC-Research\CSV\loose_no_duplicates.csv')